In [2]:
#Import the necessary packages

from astropy.table import Table
from matplotlib import pyplot as plt
%matplotlib inline
import os
import numpy as np
from astropy.io.ascii import masked

import warnings
warnings.filterwarnings('ignore')

mycolors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [3]:
#Set the home path

%env HOME=C:\Users\USER\Documents\GitHub
homedir = os.getenv("HOME")
tabledir = homedir+'/Virgo/tables/'
plotdir = homedir+'/Virgo/plots/'
datadir = homedir+'/HTML-building/galaxy/'

env: HOME=C:\Users\USER\Documents\GitHub


In [4]:
import numpy as np
from matplotlib import pyplot as plt
import os
from astropy.table import Table
from astropy.io import ascii
import glob
from astropy.io import fits
import wget
import matplotlib.image as mpimg
from astropy.wcs import WCS
from scipy.stats import scoreatpercentile
from astropy.visualization import simple_norm
from reproject import reproject_interp
import sys
from IPython.display import clear_output


In [5]:
class GalPage():
    def __init__(self,galaxy_index=None, page_name=None, catalog=None, 
                 local_path=None, path_to_galhtml=None, LS_cutout_folder=None, 
                 fits_folder=None, homepage_name=None, test=True):
        
        self.galaxy_index = int(galaxy_index)
        self.catalog = catalog
        
        self.homepage_name = homepage_name
        self.local_path = local_path
        self.LS_cutouts = LS_cutout_folder
        self.fits_folder = fits_folder
        self.path_to_galhtml = path_to_galhtml
        
        self.RA = str(self.catalog['RA_1'][self.galaxy_index])
        self.DEC = str(self.catalog['DEC_1'][self.galaxy_index])
        self.VFID = str(self.catalog['VFID'][self.galaxy_index])
        self.objname = str(self.catalog['group_name'][self.galaxy_index])   #group name is proxy for objname; if galaxy is not in group, group_name=objname
        
        self.page_name = self.VFID+'.html'   #name of galaxy html page
        
        if not test:
            self.gal_htmlpath = self.path_to_galhtml+self.page_name
        if test:
            self.gal_htmlpath = os.getenv("HOME")+'/'+self.page_name
            
                        
        #need w3 header information and w3, w1 image data
        path_to_gal = glob.glob(self.fits_folder+self.objname+'-custom-image-W3.fits')[0]
        
        
        self.wise_im, self.wise_header = fits.getdata(path_to_w3, header=True)
        
          
        #WCS header information
        self.wcs_w3 = WCS(self.wise_header)
        

In [10]:
# Open the HTML file in write mode
with open("output.html", "w") as html:
    # Write the HTML code line by line
    html.write('<html><body>\n')
    html.write('<title>Herschel images</title>\n')
    html.write('<style type="text/css">\n')
    html.write('table, td, th {padding: 5px; text-align: center; border: 2px solid black;}\n')
    html.write('p {display: inline-block;;}\n')
    html.write('</style>\n')
    html.write('<table><tr><th>VFID</th><th>Name</th><th>Legacy image</th><th>Herschel-UnimapBlue (70microns)</th><th>Herschel-UnimapBlue (100microns)</th><th>Herschel-UnimapRed (160microns)</th><th>RA</th><th>DEC</th></tr></p>')
    galaxy = Table.read(tabledir+'/Photometrytesting.csv')
    for i in range(len(galaxy)):
        galaxy_name = str(galaxy['GALAXY'][i])
        path = datadir + '\\pipeline\\' + galaxy_name
        VFID = str(galaxy['VFID'][i])
        RA = str(galaxy['RA_MOMENT'][i])
        DEC = str(galaxy['DEC_MOMENT'][i])
        if os.path.exists(path):
            html.write('<tr><td><a href="galaxy/html/'+VFID+'-'+galaxy_name+'.html">'+VFID+'</td><td>'+galaxy_name+'</td><td><a href="galaxy/png/'+VFID+'-'+galaxy_name+'-LS.jpg"><img src="galaxy/png/'+VFID+'-'+galaxy_name+'-LS.jpg" alt="No LS data.jpg" height="auto" width="100%"></a></td><td><a href="galaxy/png/'+VFID+'-'+galaxy_name+'blue.png"><img src="galaxy/png/'+VFID+'-'+galaxy_name+'blue.png" alt="Missing file 70microns.jpg" height="auto" width="100%"></a></td><td><a href="galaxy/png/'+VFID+'-'+galaxy_name+'green.png"><img src="galaxy/png/'+VFID+'-'+galaxy_name+'green.png" alt="Missing file 100microns.jpg" height="auto" width="100%"></a></td><td><a href="galaxy/png/'+VFID+'-'+galaxy_name+'red.png"><img src="galaxy/png/'+VFID+'-'+galaxy_name+'red.png" alt="Missing file 160microns.jpg" height="auto" width="100%"></a></td><td>'+RA+'</td><td>'+DEC+'</td></tr> \n')
        else:
            continue
    html.write('</tr></table>\n')
    html.write('<br /><br />\n')
    html.write('</html></body>\n')
    html.close()
print("HTML code has been written to output.html")

HTML code has been written to output.html


In [ ]:
galaxy = Table.read(tabledir+'/Photometrytesting.csv')
for i in range(len(galaxy)):
    galaxy_name = str(galaxy['GALAXY'][i])
    path = datadir + '\\pipeline\\' + galaxy_name
    VFID = str(galaxy['VFID'][i])
    RA = str(galaxy['RA_MOMENT'][i])
    DEC = str(galaxy['DEC_MOMENT'][i])
    filepath = 'galaxy/html/'+VFID+'-'+galaxy_name+'.html'
    # Write the HTML code line by line
    with open(filepath, "w") as html:
        html.write('<html><body>\n')
        html.write('<title>Photometry data</title>\n')
        html.write('<style type="text/css">\n')
        html.write('table, td, th {padding: 5px; text-align: center; border: 2px solid black;}\n')
        html.write('p {display: inline-block;;}\n')
        html.write('</style>\n')
        html.write('<table><tr><th>VFID</th><th>Name</th><th>Legacy image</th><th>Herschel-UnimapBlue (70microns)</th><th>Herschel-UnimapBlue (100microns)</th><th>Herschel-UnimapRed (160microns)</th><th>RA</th><th>DEC</th></tr></p>')
        if os.path.exists(path):
            html.write('<tr><td><a href="galaxy/html/'+VFID+'-'+galaxy_name+'.html">'+VFID+'</td><td>'+galaxy_name+'</td><td><a href="galaxy/png/'+VFID+'-'+galaxy_name+'-LS.jpg"><img src="galaxy/png/'+VFID+'-'+galaxy_name+'-LS.jpg" alt="No LS data.jpg" height="auto" width="100%"></a></td><td><a href="galaxy/png/'+VFID+'-'+galaxy_name+'blue.png"><img src="galaxy/png/'+VFID+'-'+galaxy_name+'blue.png" alt="Missing file 70microns.jpg" height="auto" width="100%"></a></td><td><a href="galaxy/png/'+VFID+'-'+galaxy_name+'green.png"><img src="galaxy/png/'+VFID+'-'+galaxy_name+'green.png" alt="Missing file 100microns.jpg" height="auto" width="100%"></a></td><td><a href="galaxy/png/'+VFID+'-'+galaxy_name+'red.png"><img src="galaxy/png/'+VFID+'-'+galaxy_name+'red.png" alt="Missing file 160microns.jpg" height="auto" width="100%"></a></td><td>'+RA+'</td><td>'+DEC+'</td></tr> \n')
        else:
            continue
        html.write('</tr></table>\n')
        html.write('<br /><br />\n')
        html.write('</html></body>\n')
        html.close()
print("done")